# Capítulo 5 - Uso de Selenium para automatizar acciones en el navegador
___
## Ejemplo práctico
___

importamos librerías clave:

In [ ]:
! pip install selenium

In [ ]:
from selenium import webdriver  # hay que haber ejecutado `pip install selenium` para que funcione la importación

Para levantar el explorador deberá estar presente en la misma carpeta que este documento el driver (descomprimido como .exe) de la versión del navegador que queramos automatizar. Se puede bajar de aquí: https://sites.google.com/chromium.org/driver/

In [ ]:
driver = webdriver.Chrome()
driver.get("http://www.python.org")

comprobación de texto en el título:

In [ ]:
print(driver.title)
assert "Python" in driver.title

referenciar un elemento de la web:

orden de rapidez: 1.- id | 2.- name | 3.- css selector | 4.- xpath

In [ ]:
from selenium.webdriver.common.by import By

# id
tbx__search = driver.find_element(By.ID, "id-search-field") 

# name
tbx__search = driver.find_element(By.NAME, "q") 

# css selector
tbx__search = driver.find_element(By.CSS_SELECTOR, "input#id-search-field")                                     # id
tbx__search = driver.find_element(By.CSS_SELECTOR, "input.search-field")                                        # class
tbx__search = driver.find_element(By.CSS_SELECTOR, "input[placeholder='Search']")                               # otro atributo
tbx__search = driver.find_element(By.CSS_SELECTOR, "input#id-search-field.search-field[placeholder='Search']")  # más de un atributo a la vez

tbx__search = driver.find_element(By.CSS_SELECTOR, "input[placeholder^='Sea']")   # empieza con
tbx__search = driver.find_element(By.CSS_SELECTOR, "input[placeholder$='arch']")  # termina con
tbx__search = driver.find_element(By.CSS_SELECTOR, "input[placeholder*='ear']")   # contiene

# xpath
tbx__search = driver.find_element(By.XPATH, "//input[@placeholder='Search']") 

Cundo el identificador no es único se puede coger una lista de elementos con el siguiente método:

In [ ]:
lista_de_elementos = driver.find_elements(By.CSS_SELECTOR, "input#id-search-field")  # devuelve un lista

tbx__search = lista_de_elementos[0]  # en este caso es 0 porque queremos el primero, pondríamos 1 si fuera el segundo, etc.

escribir en el buscador y pulsar Enter:

In [ ]:
from selenium.webdriver.common.by import By

tbx__search = driver.find_element(By.ID, "id-search-field") 
tbx__search.clear()
tbx__search.send_keys("no va a haber resultados para esta frase")

from selenium.webdriver.common.keys import Keys  # con esta línea importamos la clase Keys de selenium, que nos hará fata a continuación

tbx__search.send_keys(Keys.RETURN)  # pulsar Enter

escribir en el buscador y clicar en 'go':

In [ ]:
tbx__search = driver.find_element(By.ID, "id-search-field") 
tbx__search.clear()
tbx__search.send_keys("tampoco va a haber resultados para esta frase")

btn__go = driver.find_element(By.ID, "submit") 
btn__go.click()

más comprobaciones:

In [ ]:
print("···")
print( driver.page_source[21500:21800] )
print("···")

assert "No results found." in driver.page_source

In [ ]:
txt__results = driver.find_element(By.CSS_SELECTOR, "#content > div > section > form > ul > p")
print(txt__results.text)

assert "No results found." in txt__results.text

cerrar el navegador:

In [ ]:
driver.quit()

## Page objects
___
vamos a transformar el portal http://www.python.org en un pageobject

In [ ]:
class PortalPage(object):
    
    # constructor
    def __init__(self, driver):
        self.driver = driver
        
        # elementos
        self.tbx__search = lambda: self.driver.find_element(By.ID, "id-search-field")
        self.btn__go = lambda: self.driver.find_element(By.ID, "submit")
    
    # métodos    
    def get(self):
        driver.get("http://www.python.org")
        return self
    
    def search(self, text):
        self.tbx__search().clear()
        self.tbx__search().send_keys(text)
        self.btn__go().click()
        
    
class ResultsPage(object):
    
    def __init__(self, driver):
        self.driver = driver
        self.txt__results = lambda: self.driver.find_element(By.CSS_SELECTOR, "#content > div > section > form > ul > p")
    
    def assert_no_results_visible(self):
        assert self.txt__results().is_displayed()
        assert "No results found." in self.txt__results().text

In [ ]:
import time

driver = webdriver.Chrome()

try:
    page__portal = PortalPage(driver).get()
    page__portal.search("algo que no existe")

    page__results = ResultsPage(driver)
    page__results.assert_no_results_visible()
    
finally:
    time.sleep(3)
    driver.close()

el flujo utilizando solo los atributos sería así:

In [ ]:
import time

driver = webdriver.Chrome()

try:
    driver.get("http://www.python.org")
    
    # Portal
    page__portal = PortalPage(driver)
    page__portal.tbx__search().clear()
    page__portal.tbx__search().send_keys("algo que no existe")
    page__portal.btn__go().click()
    
    # Results
    assert "No results found." in ResultsPage(driver).txt__results().text
    print("No results found.")
    
finally:
    time.sleep(3)
    driver.close()

otro ejemplo de uso:

dada esta web (http://the-internet.herokuapp.com/login), crear debajo un script que habrá la página se loguee y luego desloguee (incluyendo asserciones)

In [ ]:
from selenium import webdriver

driver = webdriver.Chrome()

try:
    driver.get("http://the-internet.herokuapp.com/login")
    tbx__search = driver.find_element(By.NAME, "username")
    tbx__search.send_keys("tomsmith")
    tbx__search = driver.find_element(By.NAME, "password")
    tbx__search.send_keys("SuperSecretPassword!")
    btn__login = driver.find_element(By.CSS_SELECTOR, "#login > button > i")
    btn__login.click()
    btn__logout = driver.find_element(By.CSS_SELECTOR, "#content > div > a > i")
    btn__logout.click()
finally:
    driver.quit()

usando pageobjects quedaría así:

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

class Portal(object):
    
    def __init__(self, driver):
        self.driver = driver
        self.tbx__username = lambda: self.driver.find_element(By.NAME, "username")
        self.tbx__password = lambda: self.driver.find_element(By.NAME, "password")
        self.btn__login = lambda: self.driver.find_element(By.CSS_SELECTOR, "#login > button > i")
        
    def get(self):
        self.driver.get("https://the-internet.herokuapp.com/login")
        return self
    
    def login(self, username, password):
        self.tbx__username().clear()
        self.tbx__username().send_keys(username)
        self.tbx__password().clear()
        self.tbx__password().send_keys(password)
        self.btn__login().click()
        
        
class SecureArea(object):
    
    def __init__(self, driver):
        self.driver = driver
        self.btn__logout = lambda: self.driver.find_element(By.XPATH, "//*[@id='content']/div/a")
    
    def assert_page_loaded(self):
        assert self.btn__logout().is_displayed()


driver = webdriver.Chrome()

try:
    page__portal = Portal(driver).get()
    page__portal.login("tomsmith", "SuperSecretPassword!")
    page__secure_area = SecureArea(driver)
    page__secure_area.assert_page_loaded()
    page__secure_area.btn__logout().click()
finally:
    driver.quit()